In [1]:
from nnsight import LanguageModel
PATH_TO_WEIGHTS = '/home/xuzhen/switch_sae/dictionaries/moe_lb_decoder_compose/'
from sae_auto_interp.autoencoders import load_oai_autoencoders
model = LanguageModel("/home/xuzhen/switch_sae/gpt2", device_map="auto", dispatch=True)
submodule_dict = load_oai_autoencoders(model, [8], PATH_TO_WEIGHTS)
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
from sae_auto_interp.utils import load_tokenized_data
tokens = load_tokenized_data(
    ctx_len=256,
    tokenizer=model.tokenizer, 
    dataset_repo="openwebtext2-first-30-chunks-translation-examples",
    dataset_split="train[:15%]",
    )

from sae_auto_interp.features import FeatureCache
cache = FeatureCache(model, submodule_dict, batch_size=8)
cache.run(n_tokens=10_000_000, tokens=tokens)
cache.save_splits(n_splits=5, save_dir="raw_features")

from sae_auto_interp.features import FeatureDataset
from functools import partial
from sae_auto_interp.features import pool_max_activation_windows, sample
from sae_auto_interp.config import FeatureConfig, ExperimentConfig
import torch

cfg = FeatureConfig(
    width = 131_072,
    min_examples = 200,
    max_examples = 10_000,
    example_ctx_len = 20,
    n_splits = 5
)

sample_cfg = ExperimentConfig()

modules = [f".transformer.h.{8}"]

features = {mod: torch.arange(50) for mod in modules}

dataset = FeatureDataset(
    raw_dir='raw_features',
    cfg=cfg,
    features=features,
    modules=modules,
)

constructor=partial(
    pool_max_activation_windows,
    tokens=tokens,
    cfg=cfg
)

sampler = partial(
    sample,
    cfg=sample_cfg
)

for records in dataset.load(constructor=constructor, sampler=sampler):
    print(f"Got {len(records)} records")
    break

record = records[0]

from sae_auto_interp.utils import display
print(record)
print("record feature", record.feature)
display(record, model.tokenizer, n=5)

/root/miniconda3/envs/switch_sae_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['.transformer.h.8'])


Caching features: 100%|██████████| 308/308 [00:13<00:00, 22.08it/s, Total Tokens=630,784]


Total tokens processed: 630,784


Loading .transformer.h.8: 50it [00:13,  3.72it/s]

Got 50 records
record feature .transformer.h.8_feature0
